In [11]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
import pandas as pd

In [12]:
#load data
data = pd.read_csv('merged_data.csv', encoding='utf-8')

In [16]:
data.head()

,stn_id,borrowed_hour,borrowed_day,is_holiday,borrowed_num_nearby,강수량(mm),wind_chill,stn_gu,nearby_id,borrowed_date,borrowed_num
0,ST-10,0,6,1,2,0.0,21.031237,마포구,ST-2167,20221001,2
1,ST-10,1,6,1,1,0.0,20.222173,마포구,ST-2167,20221001,2
2,ST-10,2,6,1,2,0.0,19.589061,마포구,ST-2167,20221001,1
3,ST-10,3,6,1,0,0.0,19.552158,마포구,ST-2167,20221001,0
4,ST-10,4,6,1,2,0.0,18.783459,마포구,ST-2167,20221001,0


# Total Data

In [13]:
# 데이터 전처리
X = data.drop(['stn_id', 'borrowed_date', 'borrowed_num'], axis=1)
y = data['borrowed_num']

In [14]:
# 범주형 변수를 더미 변수로 변환
X = pd.get_dummies(X, columns=['stn_gu', 'nearby_id'])


In [15]:
# 훈련 세트와 테스트 세트로 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

MemoryError: Unable to allocate 18.9 GiB for an array with shape (2102, 9647438) and data type uint8

## LightGBM (total)

In [ ]:
# LightGBM 모델 생성과 훈련
params = {
    'objective': 'regression',
    'metric': 'mse',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9
}

train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

In [ ]:
# 'early_stopping_rounds'으로 수정
model = lgb.train(params, train_data, num_boost_round=1000, valid_sets=[test_data], callbacks=[
        lgb.early_stopping(stopping_rounds=3),
    ])

MemoryError: Unable to allocate 121. GiB for an array with shape (2108, 7717950) and data type float64

In [ ]:
# 모델 평가
y_pred = model.predict(X_test, num_iteration=model.best_iteration)

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mse)

print(f'Mean Squared Error: {mse}')
print(f'Mean Absolute Error: {mae}')
print(f'Root Mean Squared Error: {rmse}')

# Sample Data

In [17]:
# 필요한 열 선택
selected_features = ['stn_id', 'borrowed_hour', 'borrowed_day', 'is_holiday', 'borrowed_num_nearby', '강수량(mm)', 'wind_chill', 'stn_gu', 'nearby_id', 'borrowed_date', 'borrowed_num']
data = data[selected_features]

In [18]:
# 범주형 데이터를 숫자로 변환 (Label Encoding)
categorical_features = ['stn_id', 'stn_gu', 'nearby_id']
for feature in categorical_features:
    data[feature] = pd.factorize(data[feature])[0]

In [19]:
# train-test split
X_train, X_test, y_train, y_test = train_test_split(data.drop('borrowed_num', axis=1), data['borrowed_num'], test_size=0.2, random_state=42)

In [20]:
# LightGBM 모델 생성 및 훈련
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9
}

In [21]:
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

In [25]:
model = lgb.train(params, train_data, num_boost_round=1000, valid_sets=[test_data], callbacks=[
        lgb.early_stopping(stopping_rounds=3, verbose=100),
    ])

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.322341 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1136
[LightGBM] [Info] Number of data points in the train set: 7717950, number of used features: 10
[LightGBM] [Info] Start training from score 1.483498
Training until validation scores don't improve for 3 rounds
Did not meet early stopping. Best iteration is:
[1000]	valid_0's rmse: 1.98518


In [26]:
# 모델 평가
y_pred = model.predict(X_test, num_iteration=model.best_iteration)

# MSE 계산
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error (MSE): {mse}')

# MAE 계산
mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error (MAE): {mae}')

# RMSE 계산
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f'Root Mean Squared Error (RMSE): {rmse}')

Mean Squared Error (MSE): 3.940949911399397
Mean Absolute Error (MAE): 1.1485539408033907
Root Mean Squared Error (RMSE): 1.985182588932161


# Overfitting Check

In [27]:
# 필요한 열 선택
selected_features = ['stn_id', 'borrowed_hour', 'borrowed_day', 'is_holiday', 'borrowed_num_nearby', '강수량(mm)', 'wind_chill', 'stn_gu', 'nearby_id', 'borrowed_date', 'borrowed_num']
data = data[selected_features]


In [28]:
# 범주형 데이터를 숫자로 변환 (Label Encoding)
categorical_features = ['stn_id', 'stn_gu', 'nearby_id']
for feature in categorical_features:
    data[feature] = pd.factorize(data[feature])[0]


In [29]:
# train-test split
X_train, X_test, y_train, y_test = train_test_split(data.drop('borrowed_num', axis=1), data['borrowed_num'], test_size=0.2, random_state=42)


In [30]:
# LightGBM 모델 생성 및 훈련
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9
}


In [31]:
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)


In [32]:
model = lgb.train(params, train_data, num_boost_round=1000, valid_sets=[test_data, train_data], callbacks=[
        lgb.early_stopping(stopping_rounds=3, verbose=100),
    ])


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.274627 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1136
[LightGBM] [Info] Number of data points in the train set: 7717950, number of used features: 10
[LightGBM] [Info] Start training from score 1.483498
Training until validation scores don't improve for 3 rounds
Did not meet early stopping. Best iteration is:
[1000]	training's rmse: 1.97365	valid_0's rmse: 1.98518


In [33]:
# 모델 평가
y_pred_train = model.predict(X_train, num_iteration=model.best_iteration)
y_pred_test = model.predict(X_test, num_iteration=model.best_iteration)


In [34]:
# 훈련 데이터에서의 평가
mse_train = mean_squared_error(y_train, y_pred_train)
mae_train = mean_absolute_error(y_train, y_pred_train)
rmse_train = mean_squared_error(y_train, y_pred_train, squared=False)


print(f'Training MSE: {mse_train}')
print(f'Training MAE: {mae_train}')
print(f'Training RMSE: {rmse_train}')


# 검증 데이터에서의 평가
mse_test = mean_squared_error(y_test, y_pred_test)
mae_test = mean_absolute_error(y_test, y_pred_test)
rmse_test = mean_squared_error(y_test, y_pred_test, squared=False)

print(f'Test MSE: {mse_test}')
print(f'Test MAE: {mae_test}')
print(f'Test RMSE: {rmse_test}')

Training MSE: 3.8952857840571915
Training MAE: 1.1453070864914188
Training RMSE: 1.9736478368891426
Test MSE: 3.940949911399397
Test MAE: 1.1485539408033907
Test RMSE: 1.985182588932161
